In [1]:
pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 1.2 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/11.0 MB 1.2 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.0 MB 1.2 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/11.0 MB 1.2 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.0 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.0 MB 1.2 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.0 MB 1.2 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.0 MB 1.2 MB/s eta 0:00:08
   --------- ------------------------------ 2.6/11.0 MB 1.2 MB/s eta 0:00:08
   ---------- ----------------------------- 2.9/11.0 MB 1.2 MB/s eta 0:00:07
   ----------- -----


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.4 MB 1.2 MB/s eta 0:00:08
   --- ------------------------------------ 0.8/9.4 MB 1.2 MB/s eta 0:00:08
   ---- ----------------------------------- 1.0/9.4 MB 1.2 MB/s eta 0:00:07
   ----- ---------------------------------- 1.3/9.4 MB 1.2 MB/s eta 0:00:07
   ------ --------------------------------- 1.6/9.4 MB 1.2 MB/s eta 0:00:07
   ------- -------------------------------- 1.8/9.4 MB 1.2 MB/s eta 0:00:07
   -------- ------------------------------- 2.1/9.4 MB 1.2 MB/s eta 0:00:07
   ---------- ----------------------------- 2.4/9.4 MB 1.2 MB/s eta 0:00:06
   ----------- ---------------------------- 2.6/9.4 MB 1.2 MB/s eta 0:00:06
   ------------ --------------------------- 2.9/9.4 MB 1.2 MB/s eta 0:00:06
   ------------- ----------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from logging import raiseExceptions
from random import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [3]:
service = Service()
options = webdriver.ChromeOptions()
options.add_extension("adblock.crx")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.page_load_strategy = 'eager'
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
    "profile.managed_default_content_settings.javascript": 1,
    "profile.managed_default_content_settings.frame": 2
}

options.add_experimental_option("prefs", prefs)
options.set_capability("pageLoadStrategy", "eager")
options.add_experimental_option("prefs", prefs)

In [4]:
driver = webdriver.Chrome(service=service, options=options)

In [59]:
driver.get('https://www.tse.jus.br/comunicacao/noticias?year=2024&month=11&period=&formData=')

In [60]:
noticias = driver.find_elements(By.CLASS_NAME, 'news-list-title')
while (len(noticias) <= 150):
    btnMais = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.ID,
                'news-list-read-more-button'
                )
        )
    )
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 10000);")
    while(btnMais.text != 'Veja outras notícias'):
        time.sleep(0.1)    
        btnMais = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located(
                (By.ID,
                    'news-list-read-more-button'
                    )
            )
        )
            
    btnMais.click()
    noticias = driver.find_elements(By.CLASS_NAME, 'news-list-title')
    print(len(noticias))


10
20
30
40
50
60
70
80
90
100
110
116


KeyboardInterrupt: 

In [61]:
lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"', '‘', '’', '(', ')', ]
noticiasTreinamento = []
noticiasValidacao = []
aux = []
n = 0

# Para cada card
for card in noticias:
    texto = ''
    n += 1
    # Navega até o card e clica
    driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth',block:'center'});", card)

    link = card.find_element(By.XPATH, ".//ancestor::a[1]").get_attribute("href")
    driver.execute_script(f"window.open('{link}', '_blank');")

    driver.switch_to.window(driver.window_handles[-1])

    # Procura a notícia e armazena
    try:
        timeout = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='master_main']"))
        )

        noticiaParagrafos = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'p'))
        )
        noticiaParagrafos = noticiaParagrafos[:-7]

        for paragrafo in noticiaParagrafos:
            texto += (paragrafo.text.lstrip()).rstrip() + ' '
        
        texto = texto.split()
        print(n)

        aux = []
        if (random() < 0.8):
            for words in texto:
                for char in lista:
                    words = words.replace(char, '')
                aux.append(words.lower())

            aux = [palavra for palavra in aux if palavra != '']
            aux.insert(0, 'r')
            noticiasTreinamento.append(aux)
        else:
            for words in texto:
                for char in lista:
                    words = words.replace(char, '')
                aux.append(words.lower())

            aux = [palavra for palavra in aux if palavra != '']
            aux.insert(0, 'r')
            noticiasValidacao.append(aux)

    except Exception as e:
        print("erro:", e)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        continue

    except TimeoutException as t:
        print("timeout", t)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        continue

    # Fecha a janela e procura a prox noticia
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [78]:
driver.switch_to.window(driver.window_handles[0])

In [62]:
len(noticiasTreinamento)

90

In [63]:
arqOut = open('./TREINOnoticiasReais2022Nov.txt', 'w', encoding='utf-8')
for noticia in noticiasTreinamento:
    print(noticia, file=arqOut)
arqOut.close()

In [64]:
arqOut = open('./VALIDACAOnoticiasReais2022Nov.txt', 'w', encoding='utf-8')
for noticia in noticiasValidacao:
    print(noticia, file=arqOut)
arqOut.close()